<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/relato_integrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/relato integrado completo21 e 22.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)
dados

,ri,COD,SETOR,d11,VM30Abrilt+1,PL,LL,at-1,AT,t
0,0,AALR3,Saúde,0,2284491.1613,1175292,-5625,2540602,2542229,2021
1,0,AALR3,Saúde,0,2695528.7897,949696,-227810,2542229,2615454,2022
2,0,ABCB4,Financeiro,1,3483022.7998,4669463,572173,43328841,49565970,2021
3,0,ABCB4,Financeiro,1,3702729.69,5205386,800234,49565970,57408844,2022
4,0,ABEV3,Consumo não cíclico,0,228615744.72,82643031,12670968,125196579,138602483,2021
...,...,...,...,...,...,...,...,...,...,...
758,0,YBRA3,Outros,0,-,-66.608,-524.8,4.519,6.002,2022
759,0,YDUQ3,Consumo cíclico,0,4836112.84,3241584,158171,9265266,9897795,2021
760,0,YDUQ3,Consumo cíclico,0,2358681.36,2946247,-58244,9897795,9049207,2022
761,0,ZAMP3,Consumo cíclico,0,1716141.8484,1572720,-273841,3932755,3813408,2021


excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [25]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL' para números (float) e remove as linhas onde 'PL' seja negativo
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abrilt+1'] != '-']
dados = dados[dados['PL'] != '-']
dados = dados[dados['LL'] != '-']
dados = dados[dados['AT'] != '-']
dados = dados[dados['AT'] != '0']
dados = dados[dados['at-1'] != '-']
dados = dados[dados['at-1'] != '0']

#transforma as variáveis em números
dados['at-1'] = pd.to_numeric(dados['at-1'], errors='coerce')
dados['AT'] = pd.to_numeric(dados['AT'], errors='coerce')
dados['LL'] = pd.to_numeric(dados['LL'], errors='coerce')
dados['VM30Abrilt+1'] = pd.to_numeric(dados['VM30Abrilt+1'], errors='coerce')
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abrilt+1', 'PL', 'LL', 'at-1', 'AT']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")
dados


Tipo de dados da coluna VM30Abrilt+1: float64
Tipo de dados da coluna PL: float64
Tipo de dados da coluna LL: float64
Tipo de dados da coluna at-1: float64
Tipo de dados da coluna AT: float64


<ipython-input-25-c63e09b7b4cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN


,ri,COD,SETOR,d11,VM30Abrilt+1,PL,LL,at-1,AT,t
0,0,AALR3,Saúde,0,2.284491e+06,1175292.0,-5625.0,2540602.0,2542229.0,2021
1,0,AALR3,Saúde,0,2.695529e+06,949696.0,-227810.0,2542229.0,2615454.0,2022
4,0,ABEV3,Consumo não cíclico,0,2.286157e+08,82643031.0,12670968.0,125196579.0,138602483.0,2021
5,0,ABEV3,Consumo não cíclico,0,2.229630e+08,81955633.0,14457943.0,138602483.0,137958083.0,2022
6,0,AERI3,Bens industriais,0,3.720090e+06,999592.0,69193.0,2370629.0,3088179.0,2021
...,...,...,...,...,...,...,...,...,...,...
756,0,WLMM4,Bens industriais,0,7.129722e+05,622011.0,117298.0,668104.0,811565.0,2022
759,0,YDUQ3,Consumo cíclico,0,4.836113e+06,3241584.0,158171.0,9265266.0,9897795.0,2021
760,0,YDUQ3,Consumo cíclico,0,2.358681e+06,2946247.0,-58244.0,9897795.0,9049207.0,2022
761,0,ZAMP3,Consumo cíclico,0,1.716142e+06,1572720.0,-273841.0,3932755.0,3813408.0,2021


ESCALONAMENTO DAS VARIÁVEIS

In [1]:
variaveis = ['VM30Abrilt+1', 'PL', 'LL']

for coluna in variaveis:
    dados[coluna] = dados[coluna] / dados['at-1']


NameError: ignored

WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [26]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abrilt+1', 'PL', 'LL', 'at-1', 'AT']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


OLS

In [41]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white

# Suponha que você já tenha seu DataFrame 'dados' com as variáveis PL, LL, VM30Abrilt+1, t e ri.

# Vamos criar variáveis de interação entre 't' e 'ri' para calcular o efeito DID.
dados['t_ri'] = dados['t'] * dados['ri']

model = sm.OLS(dados['VM30Abrilt+1'], sm.add_constant(dados[['PL', 'LL', 'ri', 't', 't_ri']])).fit()

# Visualize os resultados do modelo.
print(model.summary())

# Calcule o MedAE
y_pred = model.predict(X)
medae = np.median(np.abs(y - y_pred))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X.drop(columns=['const']))
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Realizar o teste de heterocedasticidade de White
white_test_result = het_white(model.resid, X)
print("\nResultado do teste de heterocedasticidade de White:")
print("LM Statistic:", white_test_result[0])
print("LM p-value:", white_test_result[1])
print("F Statistic:", white_test_result[2])
print("F p-value:", white_test_result[3])


                            OLS Regression Results                            
Dep. Variable:           VM30Abrilt+1   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     358.6
Date:                Tue, 24 Oct 2023   Prob (F-statistic):          1.04e-140
Time:                        16:25:46   Log-Likelihood:                -8128.6
No. Observations:                 471   AIC:                         1.627e+04
Df Residuals:                     466   BIC:                         1.629e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.848e+09   1.43e+09      2.685      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


MQP

In [29]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Vamos criar variáveis de interação entre 't' e 'ri' para calcular o efeito DID.
dados['t_ri'] = dados['t'] * dados['ri']

# Ajuste o modelo OLS ponderado usando WLS (Mínimos Quadrados Ponderados) com pesos de Huber.
X = sm.add_constant(dados[['PL', 'LL', 'ri', 't', 't_ri']])
y = dados['VM30Abrilt+1']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear robusta usando o método WLS com pesos robustos
modelo_robusto = sm.RLM(y, X, M=sm.robust.norms.HuberT()).fit()

# Visualize os resultados do modelo robusto
print(modelo_robusto.summary())

# Calcule o MedAE
y_pred_robusto = modelo_robusto.predict(X)
medae_robusto = np.median(np.abs(y - y_pred_robusto))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled_robusto = (medae_robusto / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) robusto:", medae_robusto)
print("MedAE robusto escalado pelo valor (ou preço) em percentagem:", medae_scaled_robusto)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X)
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Calcular o coeficiente de determinação (R-squared)
sst_robusto = ((y - y.mean())**2).sum()
ssr_robusto = ((y - y_pred_robusto)**2).sum()
r_squared_robusto = 1 - (ssr_robusto / sst_robusto)

print("Coeficiente de Determinação (R-squared) do modelo robusto:", r_squared_robusto)

# Realizar o teste de heterocedasticidade de White
white_test_result_robusto = het_white(modelo_robusto.resid, X)
print("\nResultado do teste de heterocedasticidade de White para o modelo robusto:")
print("LM Statistic:", white_test_result_robusto[0])
print("LM p-value:", white_test_result_robusto[1])
print("F Statistic:", white_test_result_robusto[2])
print("F p-value:", white_test_result_robusto[3])


                    Robust linear Model Regression Results                    
Dep. Variable:           VM30Abrilt+1   No. Observations:                  471
Model:                            RLM   Df Residuals:                      466
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Tue, 24 Oct 2023                                         
Time:                        16:15:02                                         
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.953e+09   5.32e+08      3.669      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
